In [1]:
pip install chart-studio

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cufflinks

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lifelines

Note: you may need to restart the kernel to use updated packages.


In [4]:
import warnings
warnings.filterwarnings('ignore')
import chart_studio
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "none"
from plotly.subplots import make_subplots
import pandas as pd
import cufflinks as cf
from lifelines import KaplanMeierFitter
cf.go_offline()

In [5]:
data=pd.read_csv('survival.csv')
data=data.dropna()
data.head()

,ID,Name,Position,Age,Height(cm),Weight(kgs),Preferred Foot,Weak Foot,Nationality,Continent,...,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB
0,158023,L. Messi,RF,31,173.727522,72.137028,Left,4.0,Argentina,America,...,64,61,61,61,64,59,47,47,47,59
1,20801,Cristiano Ronaldo,ST,33,188.966778,83.025636,Right,4.0,Portugal,Europe,...,65,61,61,61,65,61,53,53,53,61
2,190871,Neymar Jr,LW,26,179.823225,68.053800,Right,5.0,Brazil,America,...,65,60,60,60,65,60,47,47,47,60
3,193080,De Gea,GK,27,195.062481,76.220256,Right,3.0,Spain,Europe,...,50,50,50,50,50,50,50,50,50,50
4,192985,K. De Bruyne,RCM,27,155.745200,69.868568,Right,5.0,Belgium,Europe,...,77,77,77,77,77,73,66,66,66,73


In [6]:
data['date_column'] = pd.to_datetime(data['Joined'])

In [7]:
data['year'] = pd.DatetimeIndex(data['Joined']).year
data['month'] = pd.DatetimeIndex(data['Joined']).month
data['day'] = pd.DatetimeIndex(data['Joined']).day

In [8]:
retirement = []

for i in data['Age']:
    if i > 29:
        retirement.append(1)
    else:
        retirement.append(0)
        

data['Retirement'] = retirement

In [9]:
data['Year'] = pd.DatetimeIndex(data['Contract_Expiry']).year

In [10]:
finish = []

for i in data['Year']:
        if i > 2022:
            finish.append(2043 - i)
        else:
            finish.append(2043 - i)
        

data['Expire'] = finish

In [11]:
null_cols=data.isnull().sum()
null_cols[null_cols>0]

Series([], dtype: int64)

In [12]:
def survival(data, group_field, time_field, event_field):
    model = KaplanMeierFitter()
    results = []
    
    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        model.fit(T, E, label=str(i))
        results.append(model.survival_function_)
    
    survival = pd.concat(results, axis=1)
    return survival

In [13]:
#data_pos = data.copy()

In [14]:
#positions = {
#    'Forward' : ['CF', 'LF', 'LS', 'LW', 'RF', 'RS', 'RW', 'ST'],
#    'Midfield' : ['CAM', 'CDM', 'CM', 'LAM', 'LCM', 'LDM', 'LM', 'RAM', 'RCB', 'RCM', 'RDM', 'RM'],
#    'Defense' : ['CB', 'LB', 'LCB', 'LWB', 'RB', 'RCB', 'RWB'],
#    'Goalkeeper' : ['GK']}

In [15]:
#def positions_counts(df):  
#    positions = {
#    'Forward' : ['CF', 'LF', 'LS', 'LW', 'RF', 'RS', 'RW', 'ST'],
#    'Midfield' : ['CAM', 'CDM', 'CM', 'LAM', 'LCM', 'LDM', 'LM', 'RAM', 'RCB', 'RCM', 'RDM', 'RM'],
#    'Defense' : ['CB', 'LB', 'LCB', 'LWB', 'RB', 'RCB', 'RWB'],
#    'Goalkeeper' : ['GK']}
    
#    pos_dic = {'Forward':[], 'Midfield':[], 'Defense':[], 'Goalkeeper':[]}
#    for i in positions:
#        pos_dic[i].append(df.loc[positions[i]].count()['ID'])
#    return pos_dic

In [16]:
#total_pos = positions_counts(data_pos)
#total_pos

In [17]:
positions = {
    'Forward' : ['CF', 'LF', 'LS', 'LW', 'RF', 'RS', 'RW', 'ST'],
    'Midfield' : ['CAM', 'CDM', 'CM', 'LAM', 'LCM', 'LDM', 'LM', 'RAM', 'RCB', 'RCM', 'RDM', 'RM'],
    'Defense' : ['CB', 'LB', 'LCB', 'LWB', 'RB', 'RCB', 'RWB'],
    'Goalkeeper' : ['GK']}

In [18]:
positions_gen = []
for e in data['Position']:
    if e in positions['Forward']:
        positions_gen.append('Forward')
    elif e in positions['Midfield']:
        positions_gen.append('Midfield')
    elif e in positions['Defense']:
        positions_gen.append('Defense')
    else:
        positions_gen.append('Goalkeeper')

In [19]:
data['Generic'] = positions_gen

In [20]:
test = data[(data['Age']>=30) & (data['Retirement']==1)]
test

,ID,Name,Position,Age,Height(cm),Weight(kgs),Preferred Foot,Weak Foot,Nationality,Continent,...,RCB,RB,date_column,year,month,day,Retirement,Year,Expire,Generic
0,158023,L. Messi,RF,31,173.727522,72.137028,Left,4.0,Argentina,America,...,47,59,2004-07-01,2004,7,1,1,2021,22,Forward
1,20801,Cristiano Ronaldo,ST,33,188.966778,83.025636,Right,4.0,Portugal,Europe,...,53,61,2018-07-10,2018,7,10,1,2022,21,Forward
6,177003,L. Modrić,RCM,32,176.775373,66.239032,Right,4.0,Croatia,Europe,...,71,79,2012-08-01,2012,8,1,1,2020,23,Midfield
7,176580,L. Suárez,RS,31,182.871076,86.201480,Right,4.0,Uruguay,America,...,63,66,2014-07-11,2014,7,11,1,2021,22,Forward
8,155862,Sergio Ramos,RCB,32,182.871076,82.118252,Right,3.0,Spain,Europe,...,87,84,2005-08-01,2005,8,1,1,2020,23,Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17857,243161,P. Azizi,CB,30,155.440415,78.035024,Right,3.0,Sweden,Europe,...,50,47,2011-02-18,2011,2,18,1,2019,24,Defense
17866,232676,N. Hatta,GK,32,185.918927,76.220256,Right,3.0,Japan,Asia,...,50,50,2005-01-01,2005,1,1,1,2021,22,Goalkeeper
18043,237917,A. Suzuki,GK,31,182.871076,79.849792,Right,2.0,Japan,Asia,...,50,50,2017-01-06,2017,1,6,1,2021,22,Goalkeeper
18171,237746,Y. Uchimura,CAM,33,179.823225,67.146416,Right,3.0,Japan,Asia,...,43,43,2010-01-01,2010,1,1,1,2021,22,Midfield


In [21]:
rates=survival(test, 'Generic', 'Age', 'Retirement')
rates.iplot(kind='line', xTitle='Años)' ,yTitle='Tasa de retiro', title='Años de carrera profesional por posición antes del retiro')

In [22]:
rates=survival(test, 'Generic', 'Expire', 'Retirement')
rates.iplot(kind='line', xTitle='Años', 
            yTitle='Tasa de retiro', title='Años de carrera profesional por años de contrato y posición')

In [23]:
rates=survival(test, 'Age', 'Expire', 'Retirement')
rates.iplot(kind='line', xTitle='Cantidad de retiros', 
            yTitle='Tasa de retiro', title='Edad más frecuente para anunciar el retiro profesional')

In [24]:
rates=survival(test, 'Nationality', 'Expire', 'Retirement')
rates.iplot(kind='line', xTitle='Cantidad de retiros', 
            yTitle='Tasa de retiro', title='Retiros profesionales por nacionalidad')

In [25]:
rates=survival(test, 'Continent', 'Expire', 'Retirement')
rates.iplot(kind='line', xTitle='Cantidad de retiros', 
            yTitle='Tasa de retiro', title='Años de carrera profesional por continente')

In [26]:
rates=survival(test, 'Preferred Foot', 'Age', 'Retirement')
rates.iplot(kind='line', xTitle='Años', 
            yTitle='Tasa de retiro', title='Años de carrera profesional según lateralidad dominante')

In [27]:
rates=survival(test, 'Height(cm)', 'Age', 'Retirement')
rates.iplot(kind='line', xTitle='Años', 
            yTitle='Tasa de retiro', title='Años de carrera profesional según la estatura')

In [28]:
rates=survival(test, 'Weight(kgs)', 'Age', 'Retirement')
rates.iplot(kind='line', xTitle='Años', 
            yTitle='Tasa de retiro', title='Años de carrera profesional según el peso')